In [1]:
import pandas as pd

def load_data(file_path):
    """Load data from a CSV file into a pandas DataFrame."""
    try:
        return pd.read_csv(file_path)
    except Exception as e:
        print(f"Error loading data: {e}")
        return None

def convert_data_types(df, date_cols=[], numeric_cols=[], categorical_cols=[]):
    """Convert columns to appropriate data types."""
    for col in date_cols:
        df[col] = pd.to_datetime(df[col], errors='coerce')
    for col in numeric_cols:
        df[col] = pd.to_numeric(df[col], errors='coerce')
    for col in categorical_cols:
        df[col] = df[col].astype('category')
    return df

def filter_data(df, col, valid_values):
    """Filter DataFrame based on valid values in a column."""
    return df[df[col].isin(valid_values)]

def check_missing_values(df):
    """Return a DataFrame with the count of missing values in each column."""
    return df.isnull().sum()

def describe_data(df, cols):
    """Return descriptive statistics for specified columns."""
    return df[cols].describe()

Loading the dataset.<br>
<br>Initial Review: Utilize head() to view the dataset's first few rows or summary statistics. 
<br>This step helps in identifying any apparent issues with data types or missing values.

In [2]:
# Load the dataset
file_path = './data/adjusted_retail_sales_data_v2.csv'
sales_data = load_data(file_path)

if sales_data is not None:
    display(sales_data.head())

,SalesDate,ProductCategory,SalesAmount,CustomerAge,CustomerGender,CustomerLocation,ProductRatings
0,2023-10-15,Home Appliances,609,22,Male,USA,4
1,2023-09-16,Clothing,1367,22,Female,Australia,5
2,2022-09-06,Electronics,1736,22,Male,UK,2
3,2023-02-24,Female,1838,35,Clothing,India,2
4,2022-09-24,Home Appliances,1829,35,Male,UK,5


<h5>Initial Exploration : Analyzing the raw dataset.</h5>
<p>It is often beneficial to perform initial exploratory data analysis, such as using <i><b>describe()</b></i> and <i><b>checking for missing values</b></i>
<br>Before making any transformations or filtering the dataset.
<br>This approach allows you to understand the dataset in its raw form and make informed decisions about how to clean and process it.</p>

In [11]:
sales_data = convert_data_types(sales_data , date_cols=['SalesDate'] , numeric_cols=['SalesAmount','CustomerAge' , 'ProductRatings'] , categorical_cols=['ProductCategory' , 'CustomerGender','CustomerLocation'])

if sales_data is not None:
    # Initial Exploration
    print("Initial Descriptive Statistics:")
    print(sales_data.describe(include='all'))  # Include all columns in the description

     # Additional Categorical Analysis
    print("\nCategorical Columns Analysis:")
    for col in ['ProductCategory', 'CustomerGender', 'CustomerLocation']:
        print(f"\nColumn: {col}")
        print(sales_data[col].value_counts())  # Frequency count of each category
    print("\nInitial Missing Values:")
    missing_values_initial = check_missing_values(sales_data)
    print(missing_values_initial)

    # Example Conclusions from Initial Exploration
    print("\n--- Initial Exploration Conclusions ---\n")
    if missing_values_initial.sum() > 0:
        print("The dataset contains missing values that need to be addressed.")
    else:
        print("There are no missing values in the dataset.")

    numeric_columns = ['SalesAmount', 'CustomerAge','ProductRatings']
    for col in numeric_columns:
        if sales_data[col].max() / sales_data[col].std() > 3:
            print(f"The '{col}' column may contain outliers as indicated by a high max/standard deviation ratio.")


Initial Descriptive Statistics:
                         SalesDate ProductCategory   SalesAmount  CustomerAge  \
count                          997            1000  9.940000e+02  1000.000000   
unique                         NaN               4           NaN          NaN   
top                            NaN     Electronics           NaN          NaN   
freq                           NaN             357           NaN          NaN   
mean    2023-01-11 20:46:27.562688             NaN  2.867501e+03    29.177000   
min            2022-01-02 00:00:00             NaN  2.200000e+01     2.000000   
25%            2022-07-03 00:00:00             NaN  5.200000e+02    22.000000   
50%            2023-01-26 00:00:00             NaN  9.870000e+02    22.000000   
75%            2023-07-16 00:00:00             NaN  1.428000e+03    35.000000   
max            2023-12-31 00:00:00             NaN  1.875000e+06   200.000000   
std                            NaN             NaN  5.944267e+04    11.680844

<h2>Data Cleaning and Transformation Steps</h2>

<p><strong>1. Handling Missing Values:</strong></p>
<ul>
  <li><em>SalesAmount</em>: Impute missing values with the median or remove rows if missing values are not randomly distributed.</li>
  <li><em>ProductRatings</em>: Impute missing values with the median or remove rows.</li>
</ul>

<p><strong>2. Addressing Outliers:</strong></p>
<ul>
  <li><em>SalesAmount</em>: Identify and handle outliers using methods like IQR. Options include capping, replacing, or removing these values.</li>
  <li><em>ProductRatings</em>: Values outside the range of 1 to 5 should be corrected or removed.</li>
</ul>

<p><strong>3. Correcting Data Types:</strong></p>
<ul>
  <li>Convert <em>SalesDate</em> to datetime format.</li>
  <li>Ensure <em>SalesAmount</em> and <em>ProductRatings</em> are numeric.</li>
</ul>

<p><strong>4. Ensuring Data Consistency:</strong></p>
<ul>
  <li>Standardize categories in <em>ProductCategory</em>, <em>CustomerGender</em>, and <em>CustomerLocation</em>.</li>
</ul>

<p><strong>5. Filtering Inconsistent Data:</strong></p>
<ul>
  <li>Remove rows with invalid categories in <em>ProductCategory</em>, <em>CustomerGender</em>, or <em>CustomerLocation</em>.</li>
</ul>

<p><strong>6. Feature Engineering (if applicable):</strong></p>
<ul>
  <li>Create new features like month or year from <em>SalesDate</em>.</li>
</ul>


In [49]:
# ... [Code for data cleaning and transformation]
if sales_data is not None:
        # Assuming sales_data is already loaded and initial exploration is done
    
    # Handling Missing Values
    sales_data['SalesAmount'].fillna(sales_data['SalesAmount'].median(), inplace=True)
    sales_data['ProductRatings'].fillna(sales_data['ProductRatings'].median(), inplace=True)
    
    # Addressing Outliers
    Q1 = sales_data['SalesAmount'].quantile(0.25)
    Q3 = sales_data['SalesAmount'].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    sales_data = sales_data[(sales_data['SalesAmount'] >= lower_bound) & (sales_data['SalesAmount'] <= upper_bound)]
    
    # Correcting Data Types and Ensuring Data Consistency
    sales_data = convert_data_types(sales_data, date_cols=['SalesDate'], numeric_cols=['SalesAmount', 'ProductRatings'])
    
    # Feature Engineering
    sales_data['Month'] = sales_data['SalesDate'].dt.month
    sales_data['Year'] = sales_data['SalesDate'].dt.year
    
    # Filtering Inconsistent Data
    sales_data = filter_data(sales_data, 'ProductCategory', ['Clothing', 'Electronics', 'Home Appliances'])
    sales_data = filter_data(sales_data, 'CustomerGender', ['Male', 'Female', 'Non-binary'])
    sales_data = filter_data(sales_data, 'CustomerLocation', ['Japan', 'Australia', 'India', 'USA', 'UK', 'Canada'])
    
    # Final check for missing values and outliers
    print("\nMissing Values After Cleaning:")
    print(check_missing_values(sales_data))
    
    print("\nDescriptive Statistics After Cleaning:")
    print(describe_data(sales_data, ['SalesAmount', 'ProductRatings']))

  
 

# Exploration After Cleaning
print("\nDescriptive Statistics After Cleaning:")
print(describe_data(sales_data, ['SalesAmount', 'ProductRatings']))

print("\nMissing Values After Cleaning:")
missing_values_post_cleaning = check_missing_values(sales_data)
print(missing_values_post_cleaning)

# Example Conclusions from Post-Cleaning Exploration
print("\n--- Post-Cleaning Exploration Conclusions ---")
if missing_values_post_cleaning.sum() == 0:
    print("Missing values have been successfully addressed.")
else:
    print("There are still missing values that need further attention.")

for col in numeric_columns:
    if sales_data[col].max() / sales_data[col].std() > 3:
        print(f"Outliers in the '{col}' column may still be present after cleaning.")




Missing Values After Cleaning:
SalesDate           3
ProductCategory     0
SalesAmount         0
CustomerAge         0
CustomerGender      0
CustomerLocation    0
ProductRatings      0
Month               3
Year                3
dtype: int64

Descriptive Statistics After Cleaning:
       SalesAmount  ProductRatings
count   972.000000      972.000000
mean    983.682099        2.948560
std     535.857543        1.405238
min      22.000000        1.000000
25%     520.000000        2.000000
50%     987.000000        3.000000
75%    1428.000000        4.000000
max    1994.000000       10.000000

Descriptive Statistics After Cleaning:
       SalesAmount  ProductRatings
count   972.000000      972.000000
mean    983.682099        2.948560
std     535.857543        1.405238
min      22.000000        1.000000
25%     520.000000        2.000000
50%     987.000000        3.000000
75%    1428.000000        4.000000
max    1994.000000       10.000000

Missing Values After Cleaning:
SalesDate      

~ Checking missing values: 

 SalesDate           3
ProductCategory     0
SalesAmount         5
CustomerAge         0
CustomerGender      0
CustomerLocation    0
ProductRatings      2
dtype: int64 

~ Describe columns: 

         SalesAmount  CustomerAge  ProductRatings  ProductRatings
count  9.680000e+02   973.000000      971.000000      971.000000
mean   2.919632e+03    29.175745        2.947477        2.947477
std    6.023556e+04    11.792921        1.406290        1.406290
min    2.200000e+01     2.000000        1.000000        1.000000
25%    5.200000e+02    22.000000        2.000000        2.000000
50%    9.870000e+02    22.000000        3.000000        3.000000
75%    1.428000e+03    35.000000        4.000000        4.000000
max    1.875000e+06   200.000000       10.000000       10.000000 

